# core

> Fill in a module description here


In [98]:
# | default_exp logger

In [99]:
# | hide
import nbdev

nbdev.nbdev_export()

In [100]:
from typing import List, Dict, Any, Optional, Union
from pydantic import BaseModel, Field

from lovely_prompts.utils import max_tokens_for_model

from lovely_prompts_server.schemas import ChatPromptBase, ChatResponseBase, ChatPrompt, ChatResponse

In [101]:
import requests
import asyncio

In [102]:
class Logger:
    def __init__(
        self,
        run_server=False,
        url_base: str = None,
        api_key: Optional[str] = None,
        project: Optional[str] = None,
        provider: Optional[str] = None,
    ):
        self.url_base = url_base or "http://localhost:8000"
        self.api_key = api_key
        self.project = project
        self.provider = provider

        if run_server:
            print("Starting server...")
            import uvicorn
            from lovely_prompts_server import app

            try:
                loop = asyncio.get_running_loop()
                config = uvicorn.Config(app, host="localhost", port=8000)
                server = uvicorn.Server(config)
                loop.create_task(server.serve())
            except Exception as e:
                print(e)

                # uvicorn.run(app, host="localhost", port=8000, )

    def log_chat_prompt(
        self,
        messages: List[Dict],
        title: Optional[str] = None,
        comment: Optional[str] = None,
    ) -> int:
        prompt_data = ChatPromptBase(title=title, messages=messages, comment=comment)
        response = requests.post(f"{self.url_base}/prompts/", params={"project": self.project}, json=prompt_data.dict())
        if response.status_code != 200:
            raise Exception(f"Failed to log prompt, status code: {response.status_code}")
        return ChatPrompt(**response.json()).id

    def log_chat_response(
        self,
        prompt_id: Union[str, int],
        response: Dict,
        title: Optional[str] = None,
        comment: Optional[str] = None,
        tok_in: Optional[int] = None,
        tok_out: Optional[int] = None,
        tok_max: Optional[int] = None,
        meta: Optional[Dict] = None,
        model: Optional[str] = None,
        temperature: Optional[float] = None,
    ) -> int:
        if not tok_max:
            tok_max = max_tokens_for_model(model)

        response_data = ChatResponseBase(
            prompt_id=prompt_id,
            title=title,
            response=response,
            comment=comment,
            tok_in=tok_in,
            tok_out=tok_out,
            tok_max=tok_max,
            meta=meta,
            model=model,
            provider=self.provider,
            temperature=temperature,
        )
        response = requests.post(
            url=f"{self.url_base}/responses/", params={"project": self.project}, json=response_data.dict()
        )
        if response.status_code != 200:
            raise Exception(f"Failed to log response, status code: {response.status_code}")
        return ChatResponse(**response.json()).id

In [103]:
from dotenv import load_dotenv

load_dotenv()

True

In [104]:
logger = Logger(run_server=False, project="default", provider="openai")

In [105]:
# | eval: false
import openai

messages = [{"role": "user", "content": "Hello there"}]

prompt_id = logger.log_chat_prompt(messages, comment="This is a comment", title="The encounter")

model = "gpt-3.5-turbo"
temperature = 0.9
max_tokens = 150


response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", temperature=0.9, max_tokens=150, messages=[{"role": "user", "content": "Hello there"}]
)

# "prompt_tokens": 9,
# "completion_tokens": 9,
# "total_tokens": 18

logger.log_chat_response(
    prompt_id,
    response.choices[0].message,
    comment="This is a comment",
    title="The response",
    model=response.model,
    temperature=0.9,
    tok_in=response.usage.prompt_tokens,
    tok_out=response.usage.completion_tokens,
)

10

In [106]:
response

<OpenAIObject chat.completion id=chatcmpl-7fQybAZw3b160pVnv5TGdlQNibSfH at 0x7f4be7362c30> JSON: {
  "id": "chatcmpl-7fQybAZw3b160pVnv5TGdlQNibSfH",
  "object": "chat.completion",
  "created": 1690109501,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hi! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 9,
    "completion_tokens": 9,
    "total_tokens": 18
  }
}

In [107]:
import asyncio


def inside_async_loop():
    try:
        asyncio.get_running_loop()
        return True
    except RuntimeError:
        return False


print(inside_async_loop())  # False if not inside a running event loop

True
